In [8]:
import cv2
import mediapipe as mp
#from math import hypot
import math
from math import hypot
#from ctypes import cast, POINTER
#from comtypes import CLSCTX_ALL
#from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import numpy as np

In [28]:
cap = cv2.VideoCapture(0) #Checks for camera
 
mpHands = mp.solutions.hands #detects hand/finger
hands = mpHands.Hands()   #complete the initialization configuration of hands
mpDraw = mp.solutions.drawing_utils
 
#To access speaker through the library pycaw 
#devices = AudioUtilities.GetSpeakers()
#interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
#volume = cast(interface, POINTER(IAudioEndpointVolume))
volbar=400
volper=0

volMin,volMax = 0,100
#volume.GetVolumeRange()[:2]

angle_max = 250
k = 1.8
angle = 120

while True:
    success,img = cap.read() #If camera works capture an image
    imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) #Convert to rgb
    
    #Collection of gesture information
    results = hands.process(imgRGB) #completes the image processing.
 
    lmList = [] #empty list
    if results.multi_hand_landmarks: #list of all hands detected.
        #By accessing the list, we can get the information of each hand's corresponding flag bit
        for handlandmark in results.multi_hand_landmarks:
            for id,lm in enumerate(handlandmark.landmark): #adding counter and returning it
                # Get finger joint points
                h,w,_ = img.shape
                cx,cy = int(lm.x*w),int(lm.y*h)
                lmList.append([id,cx,cy]) #adding to the empty list 'lmList'
            mpDraw.draw_landmarks(img,handlandmark,mpHands.HAND_CONNECTIONS)
    
    if lmList != []:
        #getting the value at a point
                        #x      #y
        x0,y0 = lmList[0][1],lmList[0][2]  #wrist
        xb,yb = lmList[5][1],lmList[5][2]  #base index
        x1,y1 = lmList[4][1],lmList[4][2]  #thumb
        x2,y2 = lmList[8][1],lmList[8][2]  #index finger
        x3,y3 = lmList[12][1],lmList[12][2]
        x4,y4 = lmList[16][1],lmList[16][2]
        x5,y5 = lmList[20][1],lmList[20][2] 
        #creating circle at the tips of thumb and index finger
        cv2.circle(img,(x1,y1),13,(255,0,0),cv2.FILLED) #image #fingers #radius #rgb
        cv2.circle(img,(x2,y2),13,(255,0,0),cv2.FILLED) #image #fingers #radius #rgb
        cv2.circle(img,(int(x0 + (x1-x0)/3),int(yb + (y0-yb)/2.3)),50,(255,0,255), 6)
        
        x_hat = xb-x2
        z_hat = hypot(xb-x2,yb-y2)
        angle_hat = math.degrees(np.arcsin(x_hat/z_hat))
        angle += int(angle_hat)
        
        length = int((angle_hat*2.8/angle_max)*250)
        
        #cv2.line(img,(x1,y1),(x2,y2),(255,0,0),3)  #create a line b/w tips of index finger and thumb
        # add a dial with arrow, clockwise to increase volume, size = arrow circumference/ half+ a little bit of radius circumference
        
 
        #length = hypot(x2-x1,y2-y1) #distance b/w tips using hypotenuse
 # from numpy we find our length,by converting hand range in terms of volume range ie b/w -63.5 to 0
        vol = np.interp(length,[0,250],[volMin,volMax]) 
        volbar=np.interp(length,[0,250],[400,150])
        volper=np.interp(length,[0,250],[0,100])
        
        
        #print(vol,int(length))
        #volume.SetMasterVolumeLevel(vol, None)
        
        # Hand range 30 - 350
        # Volume range -63.5 - 0.0
        #creating volume bar for volume level 
        cv2.rectangle(img,(50,150),(85,400),(0,0,255),4) # vid ,initial position ,ending position ,rgb ,thickness
        cv2.rectangle(img,(50,int(volbar)),(85,400),(255,0,255),cv2.FILLED)
        cv2.putText(img,f"{int(volper)}%",(10,40),cv2.FONT_ITALIC,1,(0, 255, 98),3)
        #tell the volume percentage ,location,font of text,length,rgb color,thickness
    cv2.imshow('Image',img) #Show the video 
    if cv2.waitKey(1) & 0xff==ord('q'):
        break
        
cap.release()     #stop cam       
cv2.destroyAllWindows() #close window

In [10]:
cap.release()     #stop cam       
cv2.destroyAllWindows() #close window